In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [ ]:
[max_payload, min_payload]

In [ ]:
spacex_df = spacex_df.drop(columns = ['Unnamed: 0'])

In [ ]:
sites = spacex_df['Launch Site'].unique().tolist()
sites

In [21]:
#   Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div(dcc.Dropdown(id='site-dropdown',  
                                          options=[{'label': 'All Sites', 'value': 'ALL'}, 
                                                   {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                                   {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}, 
                                                   {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                   {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}],
                                          value='ALL',
                                          placeholder="place holder here",
                                          searchable=True 
                                          )),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                html.Div(dcc.RangeSlider(id='id',
                                          min=0, max=10000, step=1000,
                                          marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500'},
                                          value=[min_payload, max_payload])),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))


def get_pie_chart(entered_site):
    filtered_df = spacex_df[['Launch Site', 'class']]
    if entered_site == 'ALL':
        fig = px.pie(filtered_df[filtered_df['class'] == 1].groupby('Launch Site')['Launch Site'].count(), values='Launch Site', 
        names= spacex_df['Launch Site'].unique().tolist(), 
        title='Total Success Launches by Site')
        return fig
    else:
        fig = px.pie(filtered_df[filtered_df['Launch Site'] == entered_site].groupby('class')['class'].count(), values='class', 
        names=['Failed', 'Success'], 
        title='Total Success Launches by ' + entered_site)
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")])

def get_scatter_plot():
    fig = px.scatter(spacex_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(port = 8050)


In [ ]:
spacex_df.head()

In [ ]:
fig = px.scatter(spacex_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category')
fig